In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# *Installing Required Libraries*

In [1]:
!pip install transformers
!pip install datasets
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylin

In [ ]:
%cd '/content/drive/MyDrive/'

/content/drive/MyDrive


In [ ]:
!unzip /content/data.zip

Archive:  /content/data.zip
  inflating: data/val.csv            
  inflating: data/train.csv          
  inflating: data/test.csv           
  inflating: data/final_val.csv      
  inflating: data/final_test.csv     
  inflating: data/final_train.csv    


# *Importing libraries*

In [2]:
import json
import pandas as pd
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import torch
from datasets import Dataset
from datasets import load_dataset
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoModelForSequenceClassification
from transformers import TrainerCallback, EarlyStoppingCallback
from transformers import TrainingArguments, Trainer

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
val_df = pd.read_csv("val.csv")


train_df = train_df.drop(columns=['Num_Tokens','Num_Sentences'])
test_df = test_df.drop(columns=['Num_Tokens','Num_Sentences'])
val_df = val_df.drop(columns=['Num_Tokens','Num_Sentences'])

In [4]:
train_df['labels'].value_counts()

,count
labels,
0,2319
1,741


In [5]:
val_df['labels'].value_counts()

,count
labels,
0,774
1,247


In [6]:
test_df['labels'].value_counts()

,count
labels,
0,774
1,247


In [7]:
train_df.to_csv('final_train.csv',index=False, encoding='utf-8')
test_df.to_csv('final_test.csv',index=False, encoding='utf-8')
val_df.to_csv('final_val.csv',index=False, encoding='utf-8')

## *Dataset loading as HuggingFace Trainer*

In [8]:
dataset = load_dataset('csv', data_files={'train': "final_train.csv",
                                              'val':"final_val.csv",'test':"final_test.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['clean_text', 'labels', 'corrected_text', 'language_tags'],
        num_rows: 3060
    })
    val: Dataset({
        features: ['clean_text', 'labels', 'corrected_text', 'language_tags'],
        num_rows: 1021
    })
    test: Dataset({
        features: ['clean_text', 'labels', 'corrected_text', 'language_tags'],
        num_rows: 1021
    })
})

## Interleaved Word-Language Approach

In [10]:
def interleaved_preprocess(sentences):
    texts = sentences['corrected_text']
    tags = sentences['language_tags']

    processed_texts = []

    for text, tag in zip(texts, tags):
        words = text.split()
        tags_list = tag.split(", ")
        interleaved_text = " ".join([f"{word} [{tag}]" for word, tag in zip(words, tags_list)])
        processed_texts.append(interleaved_text)

    return {"interleaved_text_tags": processed_texts}

dataset = dataset.map(interleaved_preprocess, batched=True)

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [11]:
print(dataset['train'][0]['interleaved_text_tags'])

hindu [EN] to [HI] kuch [HI] hai [HI] par [HI] tum [HI] to [HI] pori [HI] duniya [HI] mai [HI] gandgi [HI] macha [HI] rhe [HI] ho [HI] or [EN] bat [EN] rhi [HI] bhagwe [HI] ki [HI] to [HI] tumhe [HI] aba [EN] ke [HI] abe [HI] bhagwa [HI] hi [HI] the [HI] talvar [HI] ke [HI] dar [HI] se [HI] salvaar [HI] khol [HI] di [HI] unhone [HI] isme [HI] tumhari [HI] koi [HI] galti [HI] nhi [HI] hai [HI]


## Adjacent Sentence-Language Approach

In [12]:
def adjacent_preprocess(sentences):
    texts = sentences['corrected_text']
    tags = sentences['language_tags']

    processed_texts = []

    for text, tag in zip(texts, tags):
        adjacent_text = f"{text} [{', '.join(tag.split(', '))}]"
        processed_texts.append(adjacent_text)

    return {"adjacent_text_tags": processed_texts}

dataset = dataset.map(adjacent_preprocess, batched=True)

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [13]:
dataset['train'][0]

{'clean_text': 'hindu to kuch hai par tum to pori duniya mai gandgi macha rhe ho or bat rhi bhagwe ki to tumhe aba ke abe bhagwa hi the talvar ke dar se salvar khol di unhone isme tumhari koi galti nhi hai',
 'labels': 1,
 'corrected_text': 'hindu to kuch hai par tum to pori duniya mai gandgi macha rhe ho or bat rhi bhagwe ki to tumhe aba ke abe bhagwa hi the talvar ke dar se salvaar khol di unhone isme tumhari koi galti nhi hai',
 'language_tags': 'EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, EN, EN, HI, HI, HI, HI, HI, EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI',
 'interleaved_text_tags': 'hindu [EN] to [HI] kuch [HI] hai [HI] par [HI] tum [HI] to [HI] pori [HI] duniya [HI] mai [HI] gandgi [HI] macha [HI] rhe [HI] ho [HI] or [EN] bat [EN] rhi [HI] bhagwe [HI] ki [HI] to [HI] tumhe [HI] aba [EN] ke [HI] abe [HI] bhagwa [HI] hi [HI] the [HI] talvar [HI] ke [HI] dar [HI] se [HI] salvaar [HI] khol [HI] di [HI] unhone [HI] isme [HI] tumhari [

In [14]:
dataset['val'][0]

{'clean_text': 'hindu rastra ke ruzhan miya bhai ko kast hai chalo hindu is bat pe mast hai',
 'labels': 1,
 'corrected_text': 'hindu rastra ke rujhan miya bhai ko kast hai chalo hindu is bat pe mast hai',
 'language_tags': 'EN, EN, HI, HI, HI, HI, HI, HI, HI, HI, EN, HI, EN, HI, EN, HI',
 'interleaved_text_tags': 'hindu [EN] rastra [EN] ke [HI] rujhan [HI] miya [HI] bhai [HI] ko [HI] kast [HI] hai [HI] chalo [HI] hindu [EN] is [HI] bat [EN] pe [HI] mast [EN] hai [HI]',
 'adjacent_text_tags': 'hindu rastra ke rujhan miya bhai ko kast hai chalo hindu is bat pe mast hai [EN, EN, HI, HI, HI, HI, HI, HI, HI, HI, EN, HI, EN, HI, EN, HI]'}

In [15]:
dataset['test'][0]

{'clean_text': 'jai hind , in namuno ko bat bhi karni nahi ati hai , inko pension chahiye',
 'labels': 0,
 'corrected_text': 'jai hind , in namuno ko bat bhi karni nahi ati hai , inko pension chahiye',
 'language_tags': 'HI, EN, OOV, EN, HI, HI, EN, HI, HI, HI, HI, HI, OOV, HI, EN, HI',
 'interleaved_text_tags': 'jai [HI] hind [EN] , [OOV] in [EN] namuno [HI] ko [HI] bat [EN] bhi [HI] karni [HI] nahi [HI] ati [HI] hai [HI] , [OOV] inko [HI] pension [EN] chahiye [HI]',
 'adjacent_text_tags': 'jai hind , in namuno ko bat bhi karni nahi ati hai , inko pension chahiye [HI, EN, OOV, EN, HI, HI, EN, HI, HI, HI, HI, HI, OOV, HI, EN, HI]'}

# For 5000 rows dataset with clean_text

# *Tokenization as per Hing-MBERT*

In [16]:
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-mbert")
print(len(tokenizer))
special_tokens = ["[EN]", "[HI]", "[OOV]"]
tokenizer.add_tokens(special_tokens)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

119547


3

In [17]:
len(tokenizer)

119550

In [18]:
mbert_dataset = dataset.map(
    lambda example: tokenizer(example['interleaved_text_tags'],max_length=194,padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)
mbert_dataset = mbert_dataset.remove_columns(["clean_text","corrected_text","language_tags","interleaved_text_tags","adjacent_text_tags"])
mbert_dataset.set_format("torch")

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [19]:
mbert_dataset['train'][0]

{'labels': tensor(1),
 'input_ids': tensor([   101,  19911,  11460, 119547,  10114, 119548,  15694,  10269, 119548,
          13080, 119548,  10248, 119548,  13055,  10147, 119548,  10114, 119548,
          10183,  10116, 119548,  23145,  14990, 119548,  10737, 119548,  13412,
          10162,  11210, 119548, 109664,  10113, 119548,    186,  11643, 119548,
          13173, 119548,  10345, 119547,  11519, 119547,    186,  11924, 119548,
            170,  10921,  10240,  12577, 119548,  10879, 119548,  10114, 119548,
          13055,  34927,  10112, 119548,  11357,  10113, 119547,  11163, 119548,
          11357,  10112, 119548,    170,  10921,  10240,  11037, 119548,  11520,
         119548,  10105, 119548,  13675,  16648, 119548,  11163, 119548,  12923,
         119548,  10126, 119548,  86521,  10354, 119548,    179,  32631, 119548,
          10120, 119548,  10119,  50198, 119548,  10124,  10627, 119548,  13055,
          34927,  12476, 119548,  11252,  10116, 119548,  37188,  10116, 1

In [20]:
print("Batch Decode:")
print(tokenizer.batch_decode(mbert_dataset['train'][0]['input_ids']))

Batch Decode:
['[CLS]', 'hin', '##du', '[EN]', 'to', '[HI]', 'ku', '##ch', '[HI]', 'hai', '[HI]', 'par', '[HI]', 'tu', '##m', '[HI]', 'to', '[HI]', 'por', '##i', '[HI]', 'dun', '##iya', '[HI]', 'mai', '[HI]', 'gan', '##d', '##gi', '[HI]', 'mach', '##a', '[HI]', 'r', '##he', '[HI]', 'ho', '[HI]', 'or', '[EN]', 'bat', '[EN]', 'r', '##hi', '[HI]', 'b', '##ha', '##g', '##we', '[HI]', 'ki', '[HI]', 'to', '[HI]', 'tu', '##mh', '##e', '[HI]', 'ab', '##a', '[EN]', 'ke', '[HI]', 'ab', '##e', '[HI]', 'b', '##ha', '##g', '##wa', '[HI]', 'hi', '[HI]', 'the', '[HI]', 'tal', '##var', '[HI]', 'ke', '[HI]', 'dar', '[HI]', 'se', '[HI]', 'salva', '##ar', '[HI]', 'k', '##hol', '[HI]', 'di', '[HI]', 'un', '##hone', '[HI]', 'is', '##me', '[HI]', 'tu', '##mh', '##ari', '[HI]', 'ko', '##i', '[HI]', 'galt', '##i', '[HI]', 'n', '##hi', '[HI]', 'hai', '[HI]', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '

In [21]:
cls = [tokenizer.cls_token_id]
sep = [tokenizer.sep_token_id]

input_str = dataset['train'][0]['interleaved_text_tags']
# Tokenization happens in a few steps:
input_tokens = tokenizer.tokenize(input_str)
input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
input_ids_special_tokens = cls + input_ids + sep

decoded_str = tokenizer.decode(input_ids_special_tokens)

print("start:                ", input_str)
print("tokenize:             ", input_tokens)
print("convert_tokens_to_ids:", input_ids)
print("add special tokens:   ", input_ids_special_tokens)
print("--------")
print("decode:               ", decoded_str)

start:                 hindu [EN] to [HI] kuch [HI] hai [HI] par [HI] tum [HI] to [HI] pori [HI] duniya [HI] mai [HI] gandgi [HI] macha [HI] rhe [HI] ho [HI] or [EN] bat [EN] rhi [HI] bhagwe [HI] ki [HI] to [HI] tumhe [HI] aba [EN] ke [HI] abe [HI] bhagwa [HI] hi [HI] the [HI] talvar [HI] ke [HI] dar [HI] se [HI] salvaar [HI] khol [HI] di [HI] unhone [HI] isme [HI] tumhari [HI] koi [HI] galti [HI] nhi [HI] hai [HI]
tokenize:              ['hin', '##du', '[EN]', 'to', '[HI]', 'ku', '##ch', '[HI]', 'hai', '[HI]', 'par', '[HI]', 'tu', '##m', '[HI]', 'to', '[HI]', 'por', '##i', '[HI]', 'dun', '##iya', '[HI]', 'mai', '[HI]', 'gan', '##d', '##gi', '[HI]', 'mach', '##a', '[HI]', 'r', '##he', '[HI]', 'ho', '[HI]', 'or', '[EN]', 'bat', '[EN]', 'r', '##hi', '[HI]', 'b', '##ha', '##g', '##we', '[HI]', 'ki', '[HI]', 'to', '[HI]', 'tu', '##mh', '##e', '[HI]', 'ab', '##a', '[EN]', 'ke', '[HI]', 'ab', '##e', '[HI]', 'b', '##ha', '##g', '##wa', '[HI]', 'hi', '[HI]', 'the', '[HI]', 'tal', '##var', '[HI

# *Calculating Weights for Labels with Imbalanced Dataset*

In [22]:
# Define all possible class labels
class_labels = np.unique(mbert_dataset['train']['labels'])

# Calculate class weights
labels = mbert_dataset['train']['labels']
class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=labels.numpy())
class_weights = torch.tensor(class_weights, dtype=torch.float)
print(class_weights)

tensor([0.6598, 2.0648])


In [23]:
class_labels

array([0, 1])

# **Weighted Loss Trainer for Imbalanced Datset**

In [24]:
from transformers import TrainingArguments, Trainer
# Define a custom Trainer class to include class weights
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").to(model.device)  # Ensure labels are on the same device as model
        inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Ensure all inputs are on the same device as model
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Compute weighted loss
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))  # Move class_weights to the same device as model
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# *Function: Compute metrics for all labels*

In [25]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # Additional metrics
    recall_micro = recall_score(labels, predictions, average='micro')
    f1_micro = f1_score(labels, predictions, average='micro')

    recall_macro = recall_score(labels, predictions, average='macro')
    f1_macro = f1_score(labels, predictions, average='macro')

    recall_positive = recall_score(labels, predictions, pos_label=1)
    f1_positive = f1_score(labels, predictions, pos_label=1)

    recall_negative = recall_score(labels, predictions, pos_label=0)
    f1_negative = f1_score(labels, predictions, pos_label=0)

    return {
        'accuracy': accuracy,
        'precision_weighted': precision,
        'recall_weighted': recall,
        'f1_weighted': f1,
        'recall_micro': recall_micro,
        'f1_micro': f1_micro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
        'recall_positive': recall_positive,
        'f1_positive': f1_positive,
        'recall_negative': recall_negative,
        'f1_negative': f1_negative
    }

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1

In [26]:
model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-mbert", num_labels=2)

model.resize_token_embeddings(len(tokenizer))

arguments = TrainingArguments(
    output_dir="sample_HingMBert_trainer_5k_itags",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    optim = 'adamw_torch',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=5e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',  # Define the metric for early stopping
    greater_is_better=False,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # Additional metrics
    recall_micro = recall_score(labels, predictions, average='micro')
    f1_micro = f1_score(labels, predictions, average='micro')

    recall_macro = recall_score(labels, predictions, average='macro')
    f1_macro = f1_score(labels, predictions, average='macro')

    recall_positive = recall_score(labels, predictions, pos_label=1)
    f1_positive = f1_score(labels, predictions, pos_label=1)

    recall_negative = recall_score(labels, predictions, pos_label=0)
    f1_negative = f1_score(labels, predictions, pos_label=0)

    return {
        'accuracy': accuracy,
        'precision_weighted': precision,
        'recall_weighted': recall,
        'f1_weighted': f1,
        'recall_micro': recall_micro,
        'f1_micro': f1_micro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
        'recall_positive': recall_positive,
        'f1_positive': f1_positive,
        'recall_negative': recall_negative,
        'f1_negative': f1_negative
    }


trainer = WeightedLossTrainer(
    model=model,
    args=arguments,
    train_dataset=mbert_dataset['train'],
    eval_dataset=mbert_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/hing-mbert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# *Logger Function for early Stopping and logging*

In [26]:
class LoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
    # will call on_log on each logging step, specified by TrainerArguement. (i.e TrainerArguement.logginng_step)
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")


In [28]:
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.01))
trainer.add_callback(LoggingCallback("sample_HingMBert_trainer_5k_itags/log.jsonl"))

In [29]:
# train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Recall Micro,F1 Micro,Recall Macro,F1 Macro,Recall Positive,F1 Positive,Recall Negative,F1 Negative
1,No log,0.672684,0.755142,0.733278,0.755142,0.740226,0.755142,0.755142,0.617974,0.627933,0.352227,0.410377,0.883721,0.845488
2,No log,0.529793,0.750245,0.802421,0.750245,0.764750,0.750245,0.750245,0.748439,0.705493,0.744939,0.590690,0.751938,0.820296
3,No log,0.519138,0.761019,0.804472,0.761019,0.773783,0.761019,0.761019,0.751410,0.713722,0.732794,0.597360,0.770026,0.830084
4,No log,0.599014,0.790402,0.799709,0.790402,0.794330,0.790402,0.790402,0.734954,0.725314,0.627530,0.591603,0.842377,0.859025
5,No log,0.658819,0.775710,0.783808,0.775710,0.779271,0.775710,0.775710,0.711481,0.704203,0.587045,0.558767,0.835917,0.849639


TrainOutput(global_step=240, training_loss=0.5183963139851888, metrics={'train_runtime': 697.8409, 'train_samples_per_second': 43.85, 'train_steps_per_second': 0.688, 'train_loss': 0.5183963139851888, 'epoch': 5.0})

In [30]:
mbert_results = trainer.evaluate()

In [31]:
print(mbert_results)

{'eval_loss': 0.5191375613212585, 'eval_accuracy': 0.7610186092066601, 'eval_precision_weighted': 0.8044719591422036, 'eval_recall_weighted': 0.7610186092066601, 'eval_f1_weighted': 0.7737830895702417, 'eval_recall_micro': 0.7610186092066601, 'eval_f1_micro': 0.7610186092066601, 'eval_recall_macro': 0.7514096810302441, 'eval_f1_macro': 0.7137216507166038, 'eval_recall_positive': 0.7327935222672065, 'eval_f1_positive': 0.5973597359735974, 'eval_recall_negative': 0.7700258397932817, 'eval_f1_negative': 0.83008356545961, 'eval_runtime': 10.9384, 'eval_samples_per_second': 93.341, 'eval_steps_per_second': 5.851, 'epoch': 5.0}


# **Hing-MBERT Test results**

In [32]:
test_results_mbert = trainer.predict(mbert_dataset['test'])

In [33]:
test_results_mbert

PredictionOutput(predictions=array([[ 0.97824043, -0.6573424 ],
       [-0.21548146, -0.05820023],
       [ 1.2068838 , -1.2468514 ],
       ...,
       [-0.74744105,  0.56640875],
       [-0.9446582 ,  0.8163814 ],
       [-0.51581424,  0.3378702 ]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 0, 0]), metrics={'test_loss': 0.5004962086677551, 'test_accuracy': 0.7580803134182175, 'test_precision_weighted': 0.809696376101861, 'test_recall_weighted': 0.7580803134182175, 'test_f1_weighted': 0.7721302237143725, 'test_recall_micro': 0.7580803134182175, 'test_f1_micro': 0.7580803134182175, 'test_recall_macro': 0.7591197731956606, 'test_f1_macro': 0.714732528163414, 'test_recall_positive': 0.7611336032388664, 'test_f1_positive': 0.6035313001605136, 'test_recall_negative': 0.7571059431524548, 'test_f1_negative': 0.8259337561663144, 'test_runtime': 10.9027, 'test_samples_per_second': 93.647, 'test_steps_per_second': 5.87})

In [34]:
test_results_mbert.predictions.argmax(axis=1)

array([0, 1, 0, ..., 1, 1, 1])

In [35]:
test_df['predicted_labels_5k_hingMbert'] = test_results_mbert.predictions.argmax(axis=1)

In [36]:
print(test_results_mbert.metrics)

{'test_loss': 0.5004962086677551, 'test_accuracy': 0.7580803134182175, 'test_precision_weighted': 0.809696376101861, 'test_recall_weighted': 0.7580803134182175, 'test_f1_weighted': 0.7721302237143725, 'test_recall_micro': 0.7580803134182175, 'test_f1_micro': 0.7580803134182175, 'test_recall_macro': 0.7591197731956606, 'test_f1_macro': 0.714732528163414, 'test_recall_positive': 0.7611336032388664, 'test_f1_positive': 0.6035313001605136, 'test_recall_negative': 0.7571059431524548, 'test_f1_negative': 0.8259337561663144, 'test_runtime': 10.9027, 'test_samples_per_second': 93.647, 'test_steps_per_second': 5.87}


# **Hing BERT Training**

In [57]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [54]:
## dataset pre processing and alignment
# dataset = load_dataset('csv', data_files={'train': "data/final_train.csv",
#                                               'val':"data/final_val.csv",'test':"data/final_test.csv"})

bert_tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-bert")
bert_tokenizer.add_tokens(special_tokens)
bert_dataset = dataset.map(
    lambda example: bert_tokenizer(example['interleaved_text_tags'], max_length=194, padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)


bert_dataset = bert_dataset.remove_columns(["clean_text","corrected_text","language_tags","interleaved_text_tags","adjacent_text_tags"])
bert_dataset.set_format("torch")

In [58]:
## model training
bert_model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-bert")

bert_model.resize_token_embeddings(len(bert_tokenizer))
arguments = TrainingArguments(
    output_dir="sample_Hing_Bert_trainer_5k_itags",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    optim = 'adamw_torch',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=5e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',  # Define the metric for early stopping
    greater_is_better=False,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)

bert_trainer = WeightedLossTrainer(
    model=bert_model,
    args=arguments,
    train_dataset=bert_dataset['train'],
    eval_dataset=bert_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=bert_tokenizer,
    compute_metrics=compute_metrics
)

bert_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.01))
bert_trainer.add_callback(LoggingCallback("sample_Hing_Bert_trainer_5k_itags/log.jsonl"))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/hing-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [59]:
# train the model
bert_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Recall Micro,F1 Micro,Recall Macro,F1 Macro,Recall Positive,F1 Positive,Recall Negative,F1 Negative
1,No log,0.688738,0.757101,0.655530,0.757101,0.655103,0.757101,0.757101,0.500732,0.434804,0.004049,0.008000,0.997416,0.861607
2,No log,0.626613,0.686582,0.729871,0.686582,0.702141,0.686582,0.686582,0.641669,0.620142,0.554656,0.461279,0.728682,0.779006
3,No log,0.578702,0.775710,0.770626,0.775710,0.772931,0.775710,0.775710,0.682537,0.686795,0.502024,0.519916,0.863049,0.853674
4,No log,0.600871,0.784525,0.779398,0.784525,0.781689,0.784525,0.784525,0.693864,0.698665,0.518219,0.537815,0.869509,0.859515
5,No log,0.631200,0.782566,0.781969,0.782566,0.782264,0.782566,0.782566,0.702220,0.702777,0.546559,0.548780,0.857881,0.856774


TrainOutput(global_step=240, training_loss=0.5635477066040039, metrics={'train_runtime': 595.9711, 'train_samples_per_second': 51.345, 'train_steps_per_second': 0.805, 'train_loss': 0.5635477066040039, 'epoch': 5.0})

In [60]:
bert_eval_results = bert_trainer.evaluate()
print(bert_eval_results)

{'eval_loss': 0.5787017941474915, 'eval_accuracy': 0.7757100881488737, 'eval_precision_weighted': 0.7706255696820893, 'eval_recall_weighted': 0.7757100881488737, 'eval_f1_weighted': 0.7729314957686495, 'eval_recall_micro': 0.7757100881488737, 'eval_f1_micro': 0.7757100881488737, 'eval_recall_macro': 0.6825366935526054, 'eval_f1_macro': 0.6867951319817014, 'eval_recall_positive': 0.5020242914979757, 'eval_f1_positive': 0.519916142557652, 'eval_recall_negative': 0.8630490956072352, 'eval_f1_negative': 0.8536741214057508, 'eval_runtime': 11.0384, 'eval_samples_per_second': 92.495, 'eval_steps_per_second': 5.798, 'epoch': 5.0}


# **Test results for Hing-BERT**

In [61]:
bert_results = bert_trainer.predict(bert_dataset['test'])
test_df['predicted_labels_5k_hingBert_itags'] = bert_results.predictions.argmax(axis=1)
print(bert_results.metrics)

{'test_loss': 0.5740776658058167, 'test_accuracy': 0.7855044074436827, 'test_precision_weighted': 0.7852094476522788, 'test_recall_weighted': 0.7855044074436827, 'test_f1_weighted': 0.7853561030702929, 'test_recall_micro': 0.7855044074436827, 'test_f1_micro': 0.7855044074436827, 'test_recall_macro': 0.7069144985301656, 'test_f1_macro': 0.7071996982938675, 'test_recall_positive': 0.5546558704453441, 'test_f1_positive': 0.5557809330628803, 'test_recall_negative': 0.8591731266149871, 'test_f1_negative': 0.8586184635248548, 'test_runtime': 11.2168, 'test_samples_per_second': 91.024, 'test_steps_per_second': 5.706}


In [90]:
test_df.to_csv('final_test_5k_itags.csv',index=False, encoding='utf-8')

# **Muril cased training**

In [31]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [40]:
muril_tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
print(len(muril_tokenizer))

## Adding special tokens
special_tokens = ["[EN]", "[HI]", "[OOV]"]
muril_tokenizer.add_tokens(special_tokens)
print(len(muril_tokenizer))
muril_dataset = dataset.map(
    lambda example: muril_tokenizer(example['interleaved_text_tags'], max_length=194, padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)

muril_dataset = muril_dataset.remove_columns(["clean_text","corrected_text","language_tags","interleaved_text_tags","adjacent_text_tags"])
muril_dataset.set_format("torch")

muril_model = AutoModelForSequenceClassification.from_pretrained("google/muril-base-cased",num_labels=2)

muril_model.resize_token_embeddings(len(muril_tokenizer))

arguments = TrainingArguments(
    output_dir="sample_Hing_muril_trainer_5k_itags",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy= "epoch",
    learning_rate=3e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',  # Define the metric for early stopping
    greater_is_better=False,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)

muril_trainer = WeightedLossTrainer(
    model=muril_model,
    args=arguments,
    train_dataset=muril_dataset['train'],
    eval_dataset=muril_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=muril_tokenizer,
    compute_metrics=compute_metrics
)

muril_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.01))
muril_trainer.add_callback(LoggingCallback("sample_Hing_muril_trainer_5k_itags/log.jsonl"))

197285
197288


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [41]:
muril_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Recall Micro,F1 Micro,Recall Macro,F1 Macro,Recall Positive,F1 Positive,Recall Negative,F1 Negative
1,No log,0.692800,0.758080,0.574686,0.758080,0.653765,0.758080,0.758080,0.500000,0.431198,0.000000,0.000000,1.000000,0.862396
2,No log,0.692555,0.758080,0.574686,0.758080,0.653765,0.758080,0.758080,0.500000,0.431198,0.000000,0.000000,1.000000,0.862396
3,0.692600,0.692497,0.758080,0.574686,0.758080,0.653765,0.758080,0.758080,0.500000,0.431198,0.000000,0.000000,1.000000,0.862396


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=576, training_loss=0.6927492088741727, metrics={'train_runtime': 559.4159, 'train_samples_per_second': 54.7, 'train_steps_per_second': 3.432, 'train_loss': 0.6927492088741727, 'epoch': 3.0})

In [42]:
muril_eval_results = muril_trainer.evaluate()
print(muril_eval_results)

{'eval_loss': 0.6924971342086792, 'eval_accuracy': 0.7580803134182175, 'eval_precision_weighted': 0.5746857615922628, 'eval_recall_weighted': 0.7580803134182175, 'eval_f1_weighted': 0.6537650836609474, 'eval_recall_micro': 0.7580803134182175, 'eval_f1_micro': 0.7580803134182175, 'eval_recall_macro': 0.5, 'eval_f1_macro': 0.43119777158774375, 'eval_recall_positive': 0.0, 'eval_f1_positive': 0.0, 'eval_recall_negative': 1.0, 'eval_f1_negative': 0.8623955431754875, 'eval_runtime': 10.7104, 'eval_samples_per_second': 95.328, 'eval_steps_per_second': 5.976, 'epoch': 3.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
test1_df = pd.read_csv('/content/final_test_5k_itags.csv')

In [44]:
muril_results = muril_trainer.predict(muril_dataset['test'])
test1_df['predicted_labels_5k_muril_itags'] = muril_results.predictions.argmax(axis=1)
print(muril_results.metrics)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'test_loss': 0.6922962665557861, 'test_accuracy': 0.7580803134182175, 'test_precision_weighted': 0.5746857615922628, 'test_recall_weighted': 0.7580803134182175, 'test_f1_weighted': 0.6537650836609474, 'test_recall_micro': 0.7580803134182175, 'test_f1_micro': 0.7580803134182175, 'test_recall_macro': 0.5, 'test_f1_macro': 0.43119777158774375, 'test_recall_positive': 0.0, 'test_f1_positive': 0.0, 'test_recall_negative': 1.0, 'test_f1_negative': 0.8623955431754875, 'test_runtime': 10.9288, 'test_samples_per_second': 93.423, 'test_steps_per_second': 5.856}


## XLMR training

In [45]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [50]:
xlmr_tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")
xlmr_tokenizer.add_tokens(special_tokens)

xlmr_dataset = dataset.map(
    lambda example: xlmr_tokenizer(example['interleaved_text_tags'], max_length=194, padding='max_length', truncation=True),
    batched=True,
    batch_size=16
)

xlmr_dataset = xlmr_dataset.remove_columns(["clean_text","corrected_text","language_tags","interleaved_text_tags","adjacent_text_tags"])
xlmr_dataset.set_format("torch")

xlmr_model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/xlm-roberta-base",num_labels=2)

xlmr_model.resize_token_embeddings(len(xlmr_tokenizer))

arguments = TrainingArguments(
    output_dir="sample_Hing_xlmr_trainer_5k_itags",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy= "epoch",
    learning_rate=2e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    load_best_model_at_end=True,
     metric_for_best_model='eval_loss',  # Define the metric for early stopping
    greater_is_better=False,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)

xlmr_trainer = WeightedLossTrainer(
    model=xlmr_model,
    args=arguments,
    train_dataset=xlmr_dataset['train'],
    eval_dataset=xlmr_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=xlmr_tokenizer,
    compute_metrics=compute_metrics
)

xlmr_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
xlmr_trainer.add_callback(LoggingCallback("sample_Hing_xlmr_trainer_5k_itags/log.jsonl"))

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import torch
torch.cuda.empty_cache()

In [51]:
xlmr_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Recall Micro,F1 Micro,Recall Macro,F1 Macro,Recall Positive,F1 Positive,Recall Negative,F1 Negative
1,No log,0.691106,0.758080,0.574686,0.758080,0.653765,0.758080,0.758080,0.500000,0.431198,0.000000,0.000000,1.000000,0.862396
2,No log,0.692539,0.758080,0.574686,0.758080,0.653765,0.758080,0.758080,0.500000,0.431198,0.000000,0.000000,1.000000,0.862396
3,0.696300,0.693140,0.653281,0.659881,0.653281,0.656477,0.653281,0.653281,0.537007,0.536190,0.311741,0.303150,0.762274,0.769231
4,0.696300,0.692417,0.758080,0.574686,0.758080,0.653765,0.758080,0.758080,0.500000,0.431198,0.000000,0.000000,1.000000,0.862396
5,0.696300,0.692530,0.758080,0.574686,0.758080,0.653765,0.758080,0.758080,0.500000,0.431198,0.000000,0.000000,1.000000,0.862396


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning

TrainOutput(global_step=960, training_loss=0.6981993039449056, metrics={'train_runtime': 851.5966, 'train_samples_per_second': 35.933, 'train_steps_per_second': 2.255, 'train_loss': 0.6981993039449056, 'epoch': 5.0})

In [52]:
xlmr_eval_results = xlmr_trainer.evaluate()
print(xlmr_eval_results)

{'eval_loss': 0.6911057829856873, 'eval_accuracy': 0.7580803134182175, 'eval_precision_weighted': 0.5746857615922628, 'eval_recall_weighted': 0.7580803134182175, 'eval_f1_weighted': 0.6537650836609474, 'eval_recall_micro': 0.7580803134182175, 'eval_f1_micro': 0.7580803134182175, 'eval_recall_macro': 0.5, 'eval_f1_macro': 0.43119777158774375, 'eval_recall_positive': 0.0, 'eval_f1_positive': 0.0, 'eval_recall_negative': 1.0, 'eval_f1_negative': 0.8623955431754875, 'eval_runtime': 11.0452, 'eval_samples_per_second': 92.438, 'eval_steps_per_second': 5.794, 'epoch': 5.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [53]:
xlmr_results = xlmr_trainer.predict(xlmr_dataset['test'])
test1_df['predicted_labels_5k_xlmr'] = xlmr_results.predictions.argmax(axis=1)
print(xlmr_results.metrics)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'test_loss': 0.6897827386856079, 'test_accuracy': 0.7580803134182175, 'test_precision_weighted': 0.5746857615922628, 'test_recall_weighted': 0.7580803134182175, 'test_f1_weighted': 0.6537650836609474, 'test_recall_micro': 0.7580803134182175, 'test_f1_micro': 0.7580803134182175, 'test_recall_macro': 0.5, 'test_f1_macro': 0.43119777158774375, 'test_recall_positive': 0.0, 'test_f1_positive': 0.0, 'test_recall_negative': 1.0, 'test_f1_negative': 0.8623955431754875, 'test_runtime': 11.3343, 'test_samples_per_second': 90.081, 'test_steps_per_second': 5.647}


In [54]:
test1_df.to_csv('final_test_5k_itags.csv',index=False, encoding='utf-8')